We are pretty happy with our solution. If possible we'd like to present it in the tutorium.

Lets import stuff!

In [1]:
import numpy as np
import pandas as pd

First we define the transfer frunction. In this case it is the fermi function which is also called sigmoid.

In [2]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

Define the learning rate.

In [3]:
learning_rate = 0.8

Now we define the whole thing needed for the task as class.

In [4]:
class State:
    def __init__(self, in_dim, out_dim):
        """
        Constructor for State.
        The class represents a two layer (no hidden layers) densly connected neural Network.

        :param in_dim: The amount of input neurons.
        :param out_dim: The amount of output neurons.
        """
        # State for backpropagation
        self._out = None
        self._deriv = None
        self._delta_w = None

        self._in_dim = in_dim
        self._out_dim = out_dim
        # Weights for the output layer
        # +1 for bias
        # The weights shall be between -0.5 and 0.5
        self._weights = np.random.rand(self._out_dim, self._in_dim + 1) - 0.5

    # TODO this is useless. Remove. Removing this should also reduce allocations by a lot
    def reset(self):
        self._out = None
        self._deriv = None
        self._delta_w = None

    def forward(self, input):
        """
        This function does the forward propagation.

        :param input: The input for the NN. A numpy array of dimension ``out_dim`` as given in the constructor.
        :return: The outout of the NN. A numpy array of dimension ``out_dim`` as given in the constructor.
        """
        input_with_bias = np.concatenate((input, np.ones(1)))
        # Needed for backward
        self._input = input_with_bias
        net_sum = np.dot(self._weights, input_with_bias)
        self._out = sigmoid(net_sum)
        self._deriv = self._out * (1 - self._out)

        return self._out

    def backward(self, expected):
        """
        This function does the backward propagation.
        The loss function is the mean squared error.

        :param: The expected output for the last input given to ``forward``.
        """
        in_dim_with_bias = self._in_dim + 1
        self._delta_w = np.zeros((self._out_dim, in_dim_with_bias))
        # Update the weights of perceptron
        # TODO use np.indices instead
        for i in np.arange(self._out_dim):
            self._delta_w[i,:] = learning_rate * (expected[i] - self._out[i]) * self._deriv[i] * self._input

    def update(self):
        """
        Updates the weights of the neurons.
        Must be called after calling ``forward`` and ``backward``.
        """
        self._weights += self._delta_w
        self.reset()

The task states to load a file that was not uploaded to ecampus. We selected `PA-A_training_data_01.txt` for our code.
Since the file is not a proper csv file but we want to use pandas for loading be modified it a bit.
Please put the following in `data/PA-A_training_data_01.txt`.

Load the data.

In [5]:
path = "data/PA-A_training_data_01.txt"
df = pd.read_csv(
    path,
    sep=" ",
    comment="#",
    names=["in0", "in1", "out0"],
)

Initialize the class defined above and define an stop condition for the training. (Obviously the no of training steps is not a good criterion but it is okay for now)

In [9]:
s = State(2, 1)
iterations = 5000

Lets have a look what the network outputs without any training.

In [10]:
print(s.forward(np.array([0,0])))
print(s.forward(np.array([0,1])))
print(s.forward(np.array([0,0])))
print(s.forward(np.array([0,0])))

[0.58529472]
[0.6908783]
[0.58529472]
[0.58529472]


The training loop. We frist get a sample input and the exprected output. Then we do forward propatation, backward porpagation and finally udpate the weights.

In [7]:
for _ in range(iterations):
# We just repeditly sample from the training set
    sample = df.sample()
    input = np.array(sample[["in0", "in1"]]).squeeze()
    output = np.array(sample["out0"])

    s.forward(input)
    s.backward(output)
    s.update()

In [ ]:
Lets look how the network did.

In [11]:
print(s.forward(np.array([0,0])))
print(s.forward(np.array([0,1])))
print(s.forward(np.array([0,0])))
print(s.forward(np.array([0,0])))

[0.58529472]
[0.6908783]
[0.58529472]
[0.58529472]
